In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# Raw data: contains 3 runs for the same 125 gas mixutures = 375 exposures
path_to_raw_data = '../../data/real_NO_NO2_NH3_frequency_measurement_Ir_1_25_V_275_C_2021-03-30.xlsx'
raw = pd.read_excel(path_to_raw_data)

In [4]:
raw.head()

,Exposure nr,Cycle nr,Sample nr,NO [ppm],NO2 [ppm],NH3 [ppm],Freq [Hz],Slope sensor 1 [uA/s],Slope sensor 2 [uA/s],Average sensor 1 [uA/s],Average sensor 2 [uA/s],Sensor temperature [C]
0,1,1,1,10,5,20,0.05,-18.855169,-22.588416,32.926184,27.961554,274.994683
1,1,1,2,10,5,20,0.05,-28.289268,-28.185027,25.853867,20.915297,274.980487
2,1,1,3,10,5,20,0.05,-0.390916,-0.482129,25.756138,20.794765,274.985895
3,1,1,4,10,5,20,0.05,-0.234549,-0.156366,25.697501,20.755673,275.020372
4,1,1,5,10,5,20,0.05,-0.143336,-0.247580,25.661667,20.693778,275.014964


In [31]:
print(raw.iloc[100000:100005, :].to_latex(caption = 'Sample of raw data.',
                           label = 'tab:raw-sample', 
                           column_format='cp{0.8cm}p{0.8cm}p{0.8cm}p{0.8cm}p{1.8cm}p{1.5cm}p{1.5cm}p{1.5cm}p{1.5cm}p{1.5cm}p{1.5cm}p{1.5cm}'))

\begin{table}
\centering
\caption{Sample of raw data.}
\label{tab:raw-sample}
\begin{tabular}{cp{0.8cm}p{0.8cm}p{0.8cm}p{0.8cm}p{1.8cm}p{1.5cm}p{1.5cm}p{1.5cm}p{1.5cm}p{1.5cm}p{1.5cm}p{1.5cm}}
\toprule
{} &  Exposure nr &  Cycle nr &  Sample nr &  NO [ppm] &  NO2 [ppm] &  NH3 [ppm] &  Freq [Hz] &  Slope sensor 1 [uA/s] &  Slope sensor 2 [uA/s] &  Average sensor 1 [uA/s] &  Average sensor 2 [uA/s] &  Sensor temperature [C] \\
\midrule
100000 &          105 &         1 &        161 &         5 &          5 &         40 &        5.0 &             -38.366212 &             -48.495271 &                30.241896 &                24.821197 &              275.021724 \\
100001 &          105 &         1 &        162 &         5 &          5 &         40 &        5.0 &               6.619507 &               8.521964 &                31.896773 &                26.951688 &              274.999415 \\
100002 &          105 &         1 &        163 &         5 &          5 &         40 &        5.0 & 

In [4]:
# Renaming columns
raw.rename(columns={'Exposure nr': 'exposure',
                    'Cycle nr': 'cycle',
                    'Sample nr': 'sample',
                    'NO [ppm]': 'NO',
                    'NO2 [ppm]': 'NO2',
                    'NH3 [ppm]': 'NH3',
                    'Freq [Hz]': 'freq',
                    'Slope sensor 1 [uA/s]': '1-slope',
                    'Slope sensor 2 [uA/s]': '2-slope',
                    'Average sensor 1 [uA/s]': '1-avg',
                    'Average sensor 2 [uA/s]': '2-avg',
                    'Sensor temperature [C]': 'temp'    
                   },
           inplace = True)
raw.head()

,exposure,cycle,sample,NO,NO2,NH3,freq,1-slope,2-slope,1-avg,2-avg,temp
0,1,1,1,10,5,20,0.05,-18.855169,-22.588416,32.926184,27.961554,274.994683
1,1,1,2,10,5,20,0.05,-28.289268,-28.185027,25.853867,20.915297,274.980487
2,1,1,3,10,5,20,0.05,-0.390916,-0.482129,25.756138,20.794765,274.985895
3,1,1,4,10,5,20,0.05,-0.234549,-0.156366,25.697501,20.755673,275.020372
4,1,1,5,10,5,20,0.05,-0.143336,-0.247580,25.661667,20.693778,275.014964


In [ ]:
# ex = pd.Series(foo.loc['exposure'].unique())
# NO = pd.Series(foo.loc['NO'].unique())
# NO2 = pd.Series(foo.loc['NO2'].unique())
# NH3 = pd.Series(foo.loc['NH3'].unique())
# slope1 = foo.loc['1-slope']
# avg1 = foo.loc['1-avg']

# a = pd.concat([ex, NO, NO2, NH3, slope1, avg1], axis =0, ignore_index = True)
# a

AttributeError: module 'matplotlib.cm' has no attribute 'paired'

In [5]:
# Creating the column names of the new df
foo = raw[(raw['exposure'] == 1) & (raw['cycle'] == 1)].transpose()
gasses = ['NO', 'NO2', 'NH3']
slope_names = [str(foo.loc['freq'][i])+ '-1-slope-' + str(i) for i in range(len(foo.columns))]
avg_names = [str(foo.loc['freq'][i])+ '-1-avg-' + str(i) for i in range(len(foo.columns))]
colnames = ['exposure'] + gasses + slope_names + avg_names

In [7]:
# Separating sensors
preprocessed_sensor1 = pd.DataFrame()
preprocessed_sensor2 = pd.DataFrame()

for exp in raw['exposure'].unique():
    print(f'####################EXPOSURE{exp}################')
    for cyc in raw['cycle'].unique():
        print(f'    cycle {cyc}')
        
        # Reading common rows
        temp = raw[(raw['exposure'] == exp) & (raw['cycle'] == cyc)].transpose()
        ex = pd.Series(temp.loc['exposure'].unique())
        NO = pd.Series(temp.loc['NO'].unique())
        NO2 = pd.Series(temp.loc['NO2'].unique())
        NH3 = pd.Series(temp.loc['NH3'].unique())
        
        
        # Sensor 1
        slope1 = temp.loc['1-slope']
        avg1 = temp.loc['1-avg']
        
        # Concatenating
        sensor1_row = pd.concat([ex, NO, NO2, NH3, slope1, avg1], axis =0, ignore_index = True, names= colnames)
        # Populating
        preprocessed_sensor1 = preprocessed_sensor1.append(sensor1_row, ignore_index = True)
        
        # Sensor 2
        slope2 = temp.loc['2-slope']
        avg2 = temp.loc['2-avg']
        
        # Concatenating
        sensor2_row = pd.concat([ex, NO, NO2, NH3, slope2, avg2], axis =0, ignore_index = True, names= colnames)
        # Populating
        preprocessed_sensor2 = preprocessed_sensor2.append(sensor2_row, ignore_index = True)
        


####################EXPOSURE1################
new cycle 1
new cycle 2
new cycle 3
new cycle 4
####################EXPOSURE2################
new cycle 1
new cycle 2
new cycle 3
new cycle 4
####################EXPOSURE3################
new cycle 1
new cycle 2
new cycle 3
new cycle 4
####################EXPOSURE4################
new cycle 1
new cycle 2
new cycle 3
new cycle 4
####################EXPOSURE5################
new cycle 1
new cycle 2
new cycle 3
new cycle 4
####################EXPOSURE6################
new cycle 1
new cycle 2
new cycle 3
new cycle 4
####################EXPOSURE7################
new cycle 1
new cycle 2
new cycle 3
new cycle 4
####################EXPOSURE8################
new cycle 1
new cycle 2
new cycle 3
new cycle 4
####################EXPOSURE9################
new cycle 1
new cycle 2
new cycle 3
new cycle 4
####################EXPOSURE10################
new cycle 1
new cycle 2
new cycle 3
new cycle 4
####################EXPOSURE11################
new cycle 1


new cycle 4
####################EXPOSURE88################
new cycle 1
new cycle 2
new cycle 3
new cycle 4
####################EXPOSURE89################
new cycle 1
new cycle 2
new cycle 3
new cycle 4
####################EXPOSURE90################
new cycle 1
new cycle 2
new cycle 3
new cycle 4
####################EXPOSURE91################
new cycle 1
new cycle 2
new cycle 3
new cycle 4
####################EXPOSURE92################
new cycle 1
new cycle 2
new cycle 3
new cycle 4
####################EXPOSURE93################
new cycle 1
new cycle 2
new cycle 3
new cycle 4
####################EXPOSURE94################
new cycle 1
new cycle 2
new cycle 3
new cycle 4
####################EXPOSURE95################
new cycle 1
new cycle 2
new cycle 3
new cycle 4
####################EXPOSURE96################
new cycle 1
new cycle 2
new cycle 3
new cycle 4
####################EXPOSURE97################
new cycle 1
new cycle 2
new cycle 3
new cycle 4
####################EXPOSURE98########

new cycle 2
new cycle 3
new cycle 4
####################EXPOSURE176################
new cycle 1
new cycle 2
new cycle 3
new cycle 4
####################EXPOSURE177################
new cycle 1
new cycle 2
new cycle 3
new cycle 4
####################EXPOSURE178################
new cycle 1
new cycle 2
new cycle 3
new cycle 4
####################EXPOSURE179################
new cycle 1
new cycle 2
new cycle 3
new cycle 4
####################EXPOSURE180################
new cycle 1
new cycle 2
new cycle 3
new cycle 4
####################EXPOSURE181################
new cycle 1
new cycle 2
new cycle 3
new cycle 4
####################EXPOSURE182################
new cycle 1
new cycle 2
new cycle 3
new cycle 4
####################EXPOSURE183################
new cycle 1
new cycle 2
new cycle 3
new cycle 4
####################EXPOSURE184################
new cycle 1
new cycle 2
new cycle 3
new cycle 4
####################EXPOSURE185################
new cycle 1
new cycle 2
new cycle 3
new cycle 4
####

new cycle 4
####################EXPOSURE267################
new cycle 1
new cycle 2
new cycle 3
new cycle 4
####################EXPOSURE268################
new cycle 1
new cycle 2
new cycle 3
new cycle 4
####################EXPOSURE269################
new cycle 1
new cycle 2
new cycle 3
new cycle 4
####################EXPOSURE270################
new cycle 1
new cycle 2
new cycle 3
new cycle 4
####################EXPOSURE271################
new cycle 1
new cycle 2
new cycle 3
new cycle 4
####################EXPOSURE272################
new cycle 1
new cycle 2
new cycle 3
new cycle 4
####################EXPOSURE273################
new cycle 1
new cycle 2
new cycle 3
new cycle 4
####################EXPOSURE274################
new cycle 1
new cycle 2
new cycle 3
new cycle 4
####################EXPOSURE275################
new cycle 1
new cycle 2
new cycle 3
new cycle 4
####################EXPOSURE276################
new cycle 1
new cycle 2
new cycle 3
new cycle 4
####################EXPOSURE

new cycle 2
new cycle 3
new cycle 4
####################EXPOSURE353################
new cycle 1
new cycle 2
new cycle 3
new cycle 4
####################EXPOSURE354################
new cycle 1
new cycle 2
new cycle 3
new cycle 4
####################EXPOSURE355################
new cycle 1
new cycle 2
new cycle 3
new cycle 4
####################EXPOSURE356################
new cycle 1
new cycle 2
new cycle 3
new cycle 4
####################EXPOSURE357################
new cycle 1
new cycle 2
new cycle 3
new cycle 4
####################EXPOSURE358################
new cycle 1
new cycle 2
new cycle 3
new cycle 4
####################EXPOSURE359################
new cycle 1
new cycle 2
new cycle 3
new cycle 4
####################EXPOSURE360################
new cycle 1
new cycle 2
new cycle 3
new cycle 4
####################EXPOSURE361################
new cycle 1
new cycle 2
new cycle 3
new cycle 4
####################EXPOSURE362################
new cycle 1
new cycle 2
new cycle 3
new cycle 4
####

In [12]:
# Renaming columns
# Format: <FREQUENCY>-<SENSOR_NO>-<FEATURE_NAME>-<INDEX>
cols = {i:colnames[i] for i in range(len(colnames))}
preprocessed_sensor1.rename(columns = cols, inplace = True)
preprocessed_sensor2.rename(columns = cols, inplace = True)

In [13]:
#Saving to csv
preprocessed_sensor1.to_csv('../data/sensor1.csv', index = False)
preprocessed_sensor2.to_csv('../data/sensor2.csv', index = False)

In [ ]:
# def save_split_runs_csv(raw_data, save_path, n_mixes = 125, n_runs = 3):
#     print(f'Splitting raw file into {n_runs} runs... {n_mixes} mixtures per run...\n')
    
#     for i in range(n_runs):
#         filename = str(i) + 'run_data'
#         # Mask vector for a particular run
#         low = (n_mixes*i) + 1
#         high = low + n_mixes
#         run_points = list(range(low, high))
#         run_mask = raw_data['Exposure nr'].isin(run_points)
#         run_data = raw_data[run_mask]

#         # Finally, save each run in a separate .csv file
#         run_data.to_csv(save_path + filename + '.csv', index = False)
#     print('DONE.')

In [14]:
pd.read_csv('../data/sensor1.csv')

,exposure,NO,NO2,NH3,0.05-1-slope-0,0.05-1-slope-1,0.05-1-slope-2,0.05-1-slope-3,0.05-1-slope-4,0.05-1-slope-5,...,2500.0-1-avg-230,2500.0-1-avg-231,5000.0-1-avg-232,5000.0-1-avg-233,5000.0-1-avg-234,5000.0-1-avg-235,5000.0-1-avg-236,5000.0-1-avg-237,5000.0-1-avg-238,5000.0-1-avg-239
0,1.0,10.0,5.0,20.0,-18.855169,-28.289268,-0.390916,-0.234549,-0.143336,0.026061,...,36.095859,36.118662,35.978584,35.880855,35.880855,35.867825,35.874340,35.825476,35.840135,35.845021
1,1.0,10.0,5.0,20.0,-28.979886,-9.251672,-0.247580,-0.169397,-0.218261,0.061895,...,36.115405,36.144723,35.978584,35.962296,35.919947,35.899858,35.910174,35.861309,35.884113,35.869996
2,1.0,10.0,5.0,20.0,-25.431240,-12.874158,-0.221519,-0.234549,0.026061,-0.312733,...,36.163183,36.141466,35.962296,35.926462,35.916689,35.910174,35.906916,35.903659,35.913432,35.900401
3,1.0,10.0,5.0,20.0,-30.126572,-8.196200,-0.260610,-0.091214,-0.130305,-0.182427,...,36.170784,36.164269,36.076313,35.978584,35.946008,35.942750,35.910174,35.933792,35.939493,35.900401
4,2.0,20.0,40.0,40.0,-19.506695,-27.051368,-0.456068,-0.403946,-0.208488,-0.256267,...,35.955781,35.929720,35.805930,35.760323,35.711458,35.688655,35.685397,35.675624,35.685397,35.665852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,374.0,80.0,80.0,40.0,-27.937445,-10.891346,-0.206317,0.021718,-0.130306,-0.152023,...,34.395375,34.410849,34.261813,34.184172,34.165170,34.203175,34.164084,34.132593,34.151596,34.127164
1496,375.0,20.0,80.0,5.0,-24.358394,-22.933723,-0.206316,-0.284500,-0.078183,-0.180256,...,34.395918,34.374201,34.276472,34.212948,34.193402,34.181458,34.157025,34.184172,34.134765,34.132593
1497,375.0,20.0,80.0,5.0,-28.862612,-9.827186,-0.197630,-0.065153,-0.138992,-0.206317,...,34.408406,34.428495,34.281358,34.216749,34.180372,34.180372,34.154311,34.157025,34.159740,34.131507
1498,375.0,20.0,80.0,5.0,-25.839531,-12.780772,-0.184599,-0.143336,-0.052122,-0.097729,...,34.414921,34.388860,34.333481,34.214034,34.178743,34.186887,34.190145,34.144538,34.143452,34.138023


In [ ]:
# save_split_runs_csv(raw_data = raw, save_path = '../data/')